In [1]:
import torch
from torchsummary import summary
from models.ocflow_model import OCFlowModel
from models.networks.ocflownet import OCFlowNet
#from models.networks.image_inpainting_net import SceneCompletionNet
#from models.networks.mask_flow_net import MaskFlowNet
#from models.networks.flow_occ_net import OcclusionEstimator, CostVolumeLayer, OpticalFlowEstimator
#from models.networks.feature_pyramid_net import FeaturePyramidNet
#from models.networks.warping_layer import Warping
#from models.networks.context_net import ContextNetwork
from models.data.datasets import MpiSintelClean, MpiSintelFinal, FlyingChairs
from torch.utils.data import DataLoader

In [ ]:
#pyramid_net = FeaturePyramidNet()
#summary(pyramid_net, (3,384, 1024))

In [ ]:
#occlusion_net = OcclusionEstimator(392,level =6, highest_resolution =False)
#summary(occlusion_net, (392,6,16), 12,32)

In [ ]:
#correlation_layer = CostVolumeLayer()
#cv = summary(correlation_layer, (torch.Size([196, 6,16]), torch.Size([196, 6, 16])))

In [ ]:
#opticalflow_estimator = OpticalFlowEstimator(277,level =6, highest_resolution =False)
#summary(opticalflow_estimator, (277,6,16), 12,32)

In [ ]:
#context_network = ContextNetwork(34)
#summary(context_network,(34,96,256))

In [ ]:
#maskflow_net = MaskFlowNet()
#summary(maskflow_net, (2,3,384,1024))

In [ ]:
#scene_net = SceneCompletionNet()
#summary(scene_net, (3,384,1024), device = 'cpu')

In [ ]:
#ocflow_net = OCFlowNet()
#summary(ocflow_net, (2,3,384,1024))

# Load data    

In [ ]:
dataset = MpiSintelClean(root = '/home/trung/MPI-Sintel-training/training')
dataloader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=0)
iterator = iter(dataloader)
images, flow = iterator.next()
current_frames = images[:,0, :, :, :]
next_frames = images[:, 1, :, :, :]

# Test FeaturePyramidNet backward pass

In [ ]:
pyramid_net = FeaturePyramidNet()

In [ ]:
pyramid_im1 = pyramid_net(current_frames)

In [ ]:
grad_tensors = []
for i in range(len(pyramid_im1)): 
    grad_tensors.append(torch.randn(pyramid_im1[i].size()))

**Check that gradients of all parameters in network is initialized with None:**

In [ ]:
nonzero_grad = False
for parameters in pyramid_net.parameters(): 
    if parameters.grad!=None: 
        nonzero_grad = True
print(nonzero_grad)

**After performing backward pass, gradients of all parameters in network is filled with nonzero values:**

In [ ]:
torch.autograd.backward(pyramid_im1,grad_tensors)

In [ ]:
for parameters in pyramid_net.parameters():
    print(parameters.grad)

# Test Warping backward pass 

In [ ]:
warping = Warping()

In [ ]:
feat2 = torch.randn([5,128,12,32].size(), requires_grad = True)

In [ ]:
flow_displacement = torch.randn([feat2.size()[0], 2, feat2.size()[2],feat2.size()[3]], requires_grad = True)
print(flow_displacement.size())

In [ ]:
print(feat2.grad)
print(flow_displacement.grad)

In [ ]:
warped2 = warping(feat2, flow_displacement)

In [ ]:
warped2.size()

In [ ]:
grad_tensors = torch.randn(warped2.size())

In [ ]:
nonzero_grad = False
for parameters in warping.parameters(): 
    if parameters.grad!=None: 
        nonzero_grad = True
print(nonzero_grad)

In [ ]:
torch.autograd.backward(warped2, grad_tensors)

In [ ]:
print(feat2.grad)
print(flow_displacement.grad)

# Test OcclusionEstimator backward pass 

In [ ]:
occlu_net = OcclusionEstimator(258, level=5, highest_resolution=False)

In [ ]:
feat1 = torch.randn([5,128,12,32], requires_grad = True)
occ_features_up = torch.randn([feat1.size()[0], 1, feat1.size()[2],feat1.size()[3]], requires_grad = True)
occ_mask_up = torch.randn([feat1.size()[0], 1, feat1.size()[2],feat1.size()[3]], requires_grad = True)
warped2 = torch.randn([5,128,12,32], requires_grad = True)

In [ ]:
occ_inputs = torch.cat([feat1, warped2, occ_features_up, occ_mask_up], dim=1)

In [ ]:
nonzero_grad = False
for parameters in occlu_net.parameters(): 
    if parameters.grad!=None: 
        nonzero_grad = True
print(nonzero_grad)

In [ ]:
 occ_mask, feat_up, mask_up = occlu_net(occ_inputs,24, 64)

In [ ]:
grad_tensors = [torch.randn(occ_mask.size()),torch.randn(feat_up.size()), torch.randn(mask_up.size()) ]

In [ ]:
torch.autograd.backward([ occ_mask, feat_up, mask_up],grad_tensors)

In [ ]:
for parameters in occlu_net.parameters(): 
    print(parameters.grad)

# Test CostVolumeLayer backward pass 

In [ ]:
correlation_layer = CostVolumeLayer()
feat1 = torch.randn([5,128,12,32], requires_grad = True)
warped2 = torch.randn([5,128,12,32], requires_grad = True)


In [ ]:
cv2 =correlation_layer(feat1, warped2)

In [ ]:
cv2.shape

In [ ]:
grad_tensors = torch.randn(cv2.size())

In [ ]:
torch.autograd.backward(cv2, grad_tensors)

In [ ]:
print(feat1.grad)
print(warped2.grad)

# Test OpticalFlowEstimator backward pass

In [ ]:
up_flow = torch.randn([5,2, 12,32], requires_grad = True)
up_feature =torch.randn([5,2,12,32], requires_grad = True)
cv2 = torch.randn([5,81,12,32], requires_grad = True)
feat1 = torch.randn([5, 128,12,32], requires_grad = True)

In [ ]:
estimator_input = torch.cat([cv2, feat1, up_flow, up_feature], dim=1)

In [ ]:
optical_flow_estimator = OpticalFlowEstimator(213, level=5, highest_resolution=False)

In [ ]:
flow, up_flow1, up_feature1 =optical_flow_estimator(estimator_input, 24,64)

In [ ]:
print(flow.shape)
print(up_flow1.shape)
print(up_feature1.shape)

In [ ]:
grad_tensors = [torch.randn(flow.size()), torch.randn(up_flow1.size()), torch.randn(up_feature1.size())]

In [ ]:
torch.autograd.backward([flow,up_flow1, up_feature1], grad_tensors)

In [ ]:
for parameters in optical_flow_estimator.parameters(): 
    print(parameters.grad)

# Test ContextNetwork backward pass

In [ ]:
context_network = ContextNetwork(34)

In [ ]:
flow = torch.randn([5,34,96,256], requires_grad = True)

In [ ]:
residual_flow = context_network(flow)

In [ ]:
grad_tensors= torch.randn(residual_flow.size())

In [ ]:
nonzero_grad = False
for parameters in context_network.parameters(): 
    if parameters.grad!=None: 
        nonzero_grad = True
print(nonzero_grad)

In [ ]:
torch.autograd.backward(residual_flow, grad_tensors)

In [ ]:
for parameters in context_network.parameters(): 
    print(parameters.grad)

# Test MaskFlowNet backward pass 

In [ ]:
with torch.autograd.set_detect_anomaly(True): 
    mask_flow_net = MaskFlowNet()
    F12, O_s = mask_flow_net(images)
    grad_tensors = [torch.randn(F12.size()), torch.randn(O_s.size())]
    torch.autograd.backward([F12,O_s], grad_tensors)

In [ ]:
for parameters in mask_flow_net.parameters(): 
    print(parameters.grad)

# Test SceneCompletionNet backward pass 


In [ ]:
occluded_image = torch.randn([5,3,384,1024], requires_grad = True)

In [ ]:
completion_net = SceneCompletionNet()

In [ ]:
reconstructed_image = completion_net(occluded_image)

In [ ]:
reconstructed_image.shape

In [ ]:
grad_tensors =  torch.randn([5,3,384,1024])

In [ ]:
completion_net.zero_grad()

In [ ]:
reconstructed_image.backward(grad_tensors)

In [ ]:
for parameters in completion_net.parameters(): 
    print(parameters.grad)

# Test OCFlownet backward pass

In [ ]:
ocflownet = OCFlowNet()

In [ ]:
O_s, O_h, Ic1, Iw1 = ocflownet(images)

In [ ]:
images.shape

In [ ]:
grad_tensors = [torch.randn(O_s.size()), torch.randn(Ic1.size()), torch.randn(Iw1.size())]

In [ ]:
print(O_s.grad_fn)
print(O_h.grad_fn)
print(Ic1.grad_fn)
print(Iw1.grad_fn)

In [ ]:
O_h.requires_grad

In [ ]:
torch.autograd.backward([O_s, Ic1, Iw1], grad_tensors)

# Test OCFlowModel

In [2]:
model = OCFlowModel(dataset_name = 'MpiSintelClean', root = '/home/trung/MPI-Sintel-training/training', batch_size =32, num_epoch = 1, print_every = 1)

In [3]:
model.batch_size

32

In [ ]:
model.train()

torch.Size([32, 3, 384, 1024])
torch.Size([32, 3, 384, 1024])


In [ ]:
model.dataset[0]